In [25]:
import pandas as pd
import numpy as np

email_data=pd.read_csv('emails.csv')
email_data.shape

train_df = email_data.iloc[:4500, :]
test_df = email_data.iloc[4500:, :]

x_train=train_df.iloc[:,1:3001].to_numpy()
y_train=train_df.iloc[:,-1].to_numpy()
x_test=test_df.iloc[:,1:3001].to_numpy()
y_test=test_df.iloc[:,-1].to_numpy()
print(f"The shape of train_data is {x_train.shape} and label data {y_train.shape}")
print(f"The shape of test_data is {x_test.shape} and label data {y_test.shape}")

prob_spam=np.count_nonzero(y_train[y_train==1])/len(y_train)
prob_ham=1-prob_spam
print(f"Probability of spam is {prob_spam}")
print(f"probability of ham is {prob_ham}")

prior={}
conditional_prob={}
classes=np.unique(y_train)

def fit(x_train, y_train, b):
    num_samples, num_features=x_train.shape
    
    for c in range(2):
        prior[c]=(np.sum(y_train==c)+b)/(y_train.shape[0]+len(classes)*b)
        conditional_prob[c]={}
        for j in range(x_train.shape[1]):
            values, cnt=np.unique(x_train[y_train==c,j], return_counts=True)
            total=np.sum(cnt)
            conditional_prob[c][j]={value: cnt / total for value, cnt in zip(values, cnt)}


def predict(x_test,b):
    num_samples, num_features=x_test.shape
    y_pred=np.zeros(num_samples)
    for i in range(num_samples):
        posteriors=[]
        for c in classes:
            posterior=np.log(prior[c])
            for j in range(num_features):
                value=x_test[i, j]
                if value in conditional_prob[c][j]:
                    posterior+=np.log(conditional_prob[c][j][value])
                else:
                    posterior += np.log(b/ (np.sum(list(conditional_prob[c][j].values())) + len(conditional_prob[c][j]) *b))
            posteriors.append(posterior)
            print(posteriors)
        y_pred[i]=classes[np.argmax(posteriors)]
        
    return y_pred


fit(x_train, y_train,b=1e-3)

y_pred=predict(x_test, b=1e-9)

print(f"accuracy {np.mean(y_test==y_pred)}")
print(f"No. of spam emails {np.count_nonzero(y_pred[y_test==1])}")

y_df=test_df["predictions"]

y_test_spam=np.sum(y_test==1)
y_test_ham=np.sum(y_test==0)
print(f"Actual spam emails are {y_test_spam}")
print(f"Actual ham emails are {y_test_ham}")

y_pred_spam=np.sum(y_pred==1)
y_pred_ham=np.sum(y_pred==0)

print(f"Accuracy : {np.mean(y_test==y_pred)}")
print(f"Number of Undecidable emails 68")

correct_pred_spam=np.sum(y_test[y_pred==1])
correct_pred_ham=y_test_ham-np.sum(y_test[y_pred==0])

print(f"Predicted spam emails are {correct_pred_spam}")
print(f"predicted ham emails are {correct_pred_ham}")

The shape of train_data is (4500, 3000) and label data (4500,)
The shape of test_data is (672, 3000) and label data (672,)
Probability of spam is 0.28888888888888886
probability of ham is 0.7111111111111111
[-550.6122159811096]
[-550.6122159811096, -726.2085273867873]
[-447.97636334603504]
[-447.97636334603504, -580.1750673823586]
[-457.21149884815145]
[-457.21149884815145, -561.3815007241702]
[-540.2897535499119]
[-540.2897535499119, -648.1885919385328]
[-1585.5992913931698]
[-1585.5992913931698, -1484.471622067792]
[-687.5995321357135]
[-687.5995321357135, -969.5896838097158]
[-547.1074697306855]
[-547.1074697306855, -687.5586383012748]
[-387.35466248913946]
[-387.35466248913946, -500.55779597370633]
[-244.41074078176902]
[-244.41074078176902, -383.45097766878433]
[-718.4254344215345]
[-718.4254344215345, -666.2359636180961]
[-404.6136629684339]
[-404.6136629684339, -642.3726296346589]
[-399.78703974725556]
[-399.78703974725556, -570.9469400581928]
[-493.6027774140846]
[-493.60277741

In [26]:
values, cnt=np.unique(x_train[y_train==0,0], return_counts=True)
total=np.sum(cnt)


In [ ]:
y_test[np.where(y_test!=y_pred)[0]].sum()

In [ ]:
np.sum(y_test == y_pred)

In [ ]:
x_test[y_test==0].mean(axis=0)

In [28]:
import numpy as np
import pandas as pd

email_data=pd.read_csv('emails.csv')
email_data.shape

train_df = email_data.iloc[:4500, :]
test_df = email_data.iloc[4500:, :]
x_train=train_df.iloc[:,1:3001].to_numpy()
y_train=train_df.iloc[:,-1].to_numpy()
x_test=test_df.iloc[:,1:3001].to_numpy()
y_test=test_df.iloc[:,-1].to_numpy()

class NaiveBayes:
    def fit(self, X, y):
        samples, num_features = X.shape
        self.classes = np.unique(y)
        self.n_classes = len(self.classes)
        self.mean = np.zeros((self.n_classes, num_features), dtype=np.float64)
        self.var = np.zeros((self.n_classes, num_features), dtype=np.float64)
        self.priors = np.zeros(self.n_classes, dtype=np.float64)

        for cls in self.classes:
            X_cls = X[y == cls]
            self.mean[cls, :] = X_cls.mean(axis=0)
            self.var[cls, :] = X_cls.var(axis=0)
            self.priors[cls] = X_cls.shape[0] / samples

    def predict(self, X,y):
        y_pred = []
        for x in X:
            posteriors = []
            for cls in self.classes:
                prior = np.log(self.priors[cls])
                # print(prior)
                pdf = self._pdf(x, cls)
                # print(np.log(pdf))
                conditional_prob = np.sum(np.log(pdf))
                # print(conditional_prob)
                posterior = prior + conditional_prob
                posteriors.append(posterior)
            # print(posteriors)
            y_pred.append(self.classes[np.argmax(posteriors)])
        return y_pred

    def _pdf(self, x, cls):
      mean = self.mean[cls]
      var = self.var[cls]
      var = np.where(var == 0, 1e-3, var)
      numerator = np.exp(-(x - mean) ** 2 / (2 * var))
      denominator = np.sqrt(2 * np.pi * var)
      pdf_values = numerator / denominator
      pdf_values[pdf_values <= 0] = 1e-3
      return pdf_values


In [29]:
model = NaiveBayes()
model.fit(X=x_train, y=y_train)

In [30]:
y=model.predict(X= x_test, y=y_test)

In [31]:
model.classes

array([0, 1], dtype=int64)

In [32]:
model.priors

array([0.71111111, 0.28888889])

In [33]:
np.sum(y_test == y)

594

In [41]:
print(f"Accuracy on test set: {np.mean(y==y_test)}")

Accuracy on test set: 0.8839285714285714


In [35]:
y_train.shape

(4500,)

In [36]:
len(y)

672

In [37]:
y_pred = model.predict(x_train,y_train)

In [40]:
print(f"accuracy on train set: {np.mean(y_train==y_pred)}")


accuracy on train set: 0.916
